In [54]:
import math
import random

class City:
    def __init__(self, x=None , y=None):
        self.x = x
        self.y = y
        if x is None or y is None:
            self.x = int(random.random() * 1000)
            self.y = int(random.random() * 1000)
                
    def distanceTo(self, other) -> float:
      xDistance = abs(self.getX() - other.getX())
      yDistance = abs(self.getY() - other.getY())
      distance = math.sqrt( (xDistance*xDistance) + (yDistance*yDistance) )
      return distance
        

In [55]:
class Individual:
    def __init__(self, chromosome= []):
        self.chromosome = chromosome
        self.fitness = None
    
    def add (self, city: City):
        self.chromosome.append (city)
        
    def clone (self)->Individual:
        mutant = Indiviual (self.chromosome)
        return mutant
        
    def mutate (self, mutant, times=1):
        size = len (self.chromosome)
        for i in range (times):       
            ith = int (random.random () * size)
            jth = int (random.random () * size)
            print (ith, " ", jth)
            tmp = mutant.chromosome[ith]
            mutant.chromosome[ith] = mutant.chromosome[jth]
            mutant.chromosome[jth] = tmp
            print ("m", mutant)
        
    #def crossover (self, other : Individual)->Individual:
        
    def calcFitness (self)->float:
        length = ((lambda i1, i2: i1.distanceTo(i2)), [1, 2, 3, 4])
        return length
        
    
    def __repr__(self):
        braket = "<"
        for i in range(0, len (self.chromosome)):
            braket += "(" + str(self.chromosome[i].x) + "," + str(self.chromosome[i].y)+ ")"+ "|"
        braket += ">" 
        return braket
        
        

In [56]:
c1 = City(10,20)
c2 = City(100, 200)
i1 = Individual ()
i1.add(c1)
i1.add(c2)
mutant = Individual ()
i1.mutate (mutant)
print (i1)

print (mutant)




0   1
m <(100,200)|(10,20)|>
<(100,200)|(10,20)|>
<(100,200)|(10,20)|>
